### Imports

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor
from IPython.display import display

from sklearn import metrics

In [3]:
import random
import feather

### Constants and funcs 

In [4]:
PATH = 'data/house_prices/'

In [5]:
!ls {PATH}

data_description.txt     sample_train.csv         tmp
sample_submission.csv    test.csv                 train.csv
sample_submission.csv.gz test.csv.gz              train.csv.gz


In [6]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [25]:
def rmse(x,y): return math.sqrt(((np.log(x)-np.log(y))**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [12]:
# Count the lines
num_lines = sum(1 for l in open(f'{PATH}train.csv'))

# Sample size - in this case ~10%
size = int(num_lines / 10)

# The row indices to skip - make sure 0 is not included to keep the header!
skip_idx = random.sample(range(1, num_lines), num_lines - size)

### Loads 

In [16]:
sample_df = pd.read_csv(f'{PATH}train.csv', skiprows=skip_idx)

In [8]:
raw_train_df = pd.read_csv(f'{PATH}train.csv', low_memory=False)

In [9]:
raw_test_df = pd.read_csv(f'{PATH}test.csv')

In [10]:
len(raw_train_df), len(raw_test_df)

(1460, 1459)

### EDA on sample

In [17]:
sample_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,27,20,RL,60.0,7200,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,134800
1,32,20,RL,NaN,8544,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,6,2008,WD,Normal,149350
2,49,190,RM,33.0,4456,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2009,New,Partial,113000
3,59,60,RL,66.0,13682,Pave,NaN,IR2,HLS,AllPub,...,0,NaN,NaN,NaN,0,10,2006,New,Partial,438780
4,64,70,RM,50.0,10300,Pave,NaN,IR1,Bnk,AllPub,...,0,NaN,GdPrv,NaN,0,4,2010,WD,Normal,140000


In [21]:
display_all(sample_df.describe(include='all'))

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
count,145.00000,145.000000,145,122.000000,145.000000,145,7,145,145,145,145,145,145,145,145,145,145,145.000000,145.000000,145.000000,145.000000,145,145,145,145,144,144.000000,145,145,145,142,142,142,142,145.000000,141,145.000000,145.000000,145.000000,145,145,145,145,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145,145.000000,145,145.000000,70,136,136.000000,136,145.000000,145.000000,136,136,145,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,1,33,3,145.000000,145.000000,145.000000,145,145,145.000000
unique,NaN,NaN,5,NaN,NaN,1,2,4,4,1,4,3,22,6,3,5,8,NaN,NaN,NaN,NaN,4,2,12,14,4,NaN,4,4,5,4,3,4,6,NaN,6,NaN,NaN,NaN,3,4,2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,5,NaN,5,4,NaN,3,NaN,NaN,3,2,3,NaN,NaN,NaN,NaN,NaN,NaN,1,4,2,NaN,NaN,NaN,6,6,NaN
top,NaN,NaN,RL,NaN,NaN,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,NaN,NaN,NaN,NaN,Gable,CompShg,VinylSd,VinylSd,None,NaN,TA,TA,PConc,TA,TA,No,Unf,NaN,Unf,NaN,NaN,NaN,GasA,Ex,Y,SBrkr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TA,NaN,Typ,NaN,Gd,Attchd,NaN,Unf,NaN,NaN,TA,TA,Y,NaN,NaN,NaN,NaN,NaN,NaN,Ex,MnPrv,Shed,NaN,NaN,NaN,WD,Normal,NaN
freq,NaN,NaN,110,NaN,NaN,145,6,96,127,145,103,135,21,128,142,121,80,NaN,NaN,NaN,NaN,111,144,40,39,86,NaN,96,132,65,67,131,89,50,NaN,125,NaN,NaN,NaN,143,69,138,132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78,NaN,137,NaN,37,84,NaN,71,NaN,NaN,130,131,131,NaN,NaN,NaN,NaN,NaN,NaN,1,16,2,NaN,NaN,NaN,121,113,NaN
mean,771.17931,53.586207,NaN,70.565574,10360.986207,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.096552,5.648276,1969.275862,1984.317241,NaN,NaN,NaN,NaN,NaN,104.534722,NaN,NaN,NaN,NaN,NaN,NaN,NaN,379.234483,NaN,60.717241,648.455172,1088.406897,NaN,NaN,NaN,NaN,1165.979310,315.551724,10.503448,1492.034483,0.413793,0.068966,1.565517,0.337931,2.868966,1.082759,NaN,6.496552,NaN,0.531034,NaN,NaN,1978.948529,NaN,1.820690,471.186207,NaN,NaN,NaN,90.620690,53.875862,23.848276,2.124138,10.227586,3.531034,NaN,NaN,NaN,8.275862,6.758621,2007.951724,NaN,NaN,179422.813793
std,418.68411,39.165352,NaN,23.422454,7412.202802,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.415804,1.102592,30.977026,21.106444,NaN,NaN,NaN,NaN,NaN,175.014921,NaN,NaN,NaN,NaN,NaN,NaN,NaN,410.885730,NaN,199.260323,512.833008,469.782596,NaN,NaN,NaN,NaN,396.174107,429.637605,58.767683,497.502811,0.508077,0.254274,0.575106,0.503056,0.801519,0.300542,NaN,1.704260,NaN,0.589906,NaN,NaN,24.623181,NaN,0.847127,230.406012,NaN,NaN,NaN,132.393514,90.185371,66.734167,18.098835,49.056479,42.519286,NaN,NaN,NaN,74.071520,2.677779,1.303473,NaN,NaN,84286.253844
min,27.00000,20.000000,NaN,21.000000,1680.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000,3.000000,1893.000000,1950.000000,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,483.000000,0.000000,0.000000,672.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,NaN,3.000000,NaN,0.000000,NaN,NaN,1914.000000,NaN,0.000000,0.000000,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.000000,1.000000,2006.000000

In [19]:
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145 entries, 0 to 144
Data columns (total 81 columns):
Id               145 non-null int64
MSSubClass       145 non-null int64
MSZoning         145 non-null object
LotFrontage      122 non-null float64
LotArea          145 non-null int64
Street           145 non-null object
Alley            7 non-null object
LotShape         145 non-null object
LandContour      145 non-null object
Utilities        145 non-null object
LotConfig        145 non-null object
LandSlope        145 non-null object
Neighborhood     145 non-null object
Condition1       145 non-null object
Condition2       145 non-null object
BldgType         145 non-null object
HouseStyle       145 non-null object
OverallQual      145 non-null int64
OverallCond      145 non-null int64
YearBuilt        145 non-null int64
YearRemodAdd     145 non-null int64
RoofStyle        145 non-null object
RoofMatl         145 non-null object
Exterior1st      145 non-null object
Exterior2nd    

###  Fast cleaning and preparation

In [26]:
# use `add_datepart` to add different datetime features (works with datetime columns)

In [11]:
# object dtype -> category
train_cats(raw_train_df)

In [12]:
# apply same categories as in train for test
apply_cats(raw_test_df, raw_train_df)

In [13]:
raw_train_df.MSZoning.cat.categories

Index(['C (all)', 'FV', 'RH', 'RL', 'RM'], dtype='object')

In [14]:
raw_test_df.MSZoning.cat.categories

Index(['C (all)', 'FV', 'RH', 'RL', 'RM'], dtype='object')

In [15]:
# percent of nans
display_all(raw_train_df.isnull().sum().sort_index()/len(raw_train_df))

1stFlrSF         0.000000
2ndFlrSF         0.000000
3SsnPorch        0.000000
Alley            0.937671
BedroomAbvGr     0.000000
BldgType         0.000000
BsmtCond         0.025342
BsmtExposure     0.026027
BsmtFinSF1       0.000000
BsmtFinSF2       0.000000
BsmtFinType1     0.025342
BsmtFinType2     0.026027
BsmtFullBath     0.000000
BsmtHalfBath     0.000000
BsmtQual         0.025342
BsmtUnfSF        0.000000
CentralAir       0.000000
Condition1       0.000000
Condition2       0.000000
Electrical       0.000685
EnclosedPorch    0.000000
ExterCond        0.000000
ExterQual        0.000000
Exterior1st      0.000000
Exterior2nd      0.000000
Fence            0.807534
FireplaceQu      0.472603
Fireplaces       0.000000
Foundation       0.000000
FullBath         0.000000
Functional       0.000000
GarageArea       0.000000
GarageCars       0.000000
GarageCond       0.055479
GarageFinish     0.055479
GarageQual       0.055479
GarageType       0.055479
GarageYrBlt      0.055479
GrLivArea   

In [16]:
raw_train_df.SalePrice.describe()

count      1460.000000
mean     180921.195890
std       79442.502883
min       34900.000000
25%      129975.000000
50%      163000.000000
75%      214000.000000
max      755000.000000
Name: SalePrice, dtype: float64

In [17]:
# save to feather
os.makedirs(f'{PATH}tmp', exist_ok=True)

In [18]:
raw_train_df.to_feather(f'{PATH}tmp/train.ft')
raw_test_df.to_feather(f'{PATH}tmp/test.ft')

In [19]:
# load feather
raw_train_df = feather.read_dataframe(f'{PATH}tmp/train.ft')
raw_test_df = feather.read_dataframe(f'{PATH}tmp/test.ft')

In [8]:
??proc_df

In [20]:
# label encoding and fillna with median with `proc_df`
df, y, nas = proc_df(raw_train_df, 'SalePrice', skip_flds=['Id'])

In [21]:
test, _, _ = proc_df(raw_test_df, skip_flds=['Id'], na_dict=nas)

In [10]:
m = RandomForestRegressor(n_jobs=-1, n_estimators=10)
m.fit(df, y)
m.score(df,y)

0.9759772071749353

### Make validation 

In [29]:
X_train, X_valid, y_train, y_valid = sklearn.model_selection.train_test_split(
    df, y, test_size=0.33, random_state=43)

In [30]:
X_train.shape, y_train.shape

((978, 82), (978,))

In [31]:
m = RandomForestRegressor(n_jobs=-1, n_estimators=10)
%time m.fit(X_train, y_train)
print_score(m)

CPU times: user 113 ms, sys: 2.35 ms, total: 116 ms
Wall time: 116 ms
[0.07009329398735005, 0.16791860448422335, 0.9696759387974037, 0.8288356171759014]


In [32]:
y_pred = m.predict(test)

In [33]:
y_pred.shape

(1459,)

### Submission 

In [34]:
submission = pd.read_csv(f'{PATH}sample_submission.csv')

In [35]:
submission.SalePrice.mean()

179183.91824266256

In [36]:
all(submission.Id == raw_test_df.Id)

True

In [37]:
submission.SalePrice = y_pred

In [38]:
submission.SalePrice.mean()

179001.27039067855

In [39]:
os.makedirs(f'{PATH}tmp/subm', exist_ok=True)

In [40]:
submission.to_csv(f'{PATH}tmp/subm/rf_base_submission.csv', index=False, header=True)